d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# What is Machine Learning?

Machine learning discovers patterns within data without being explicitly programmed.  This lesson introduces machine learning, explores the main topics in the field, and builds an end-to-end pipeline in Spark.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
* Define machine learning
* Differentiate supervised and unsupervised tasks
* Identify regression and classification tasks
* Train a model, interpret the results, and create predictions

<iframe  
src="//fast.wistia.net/embed/iframe/h8cqzugdrf?videoFoam=true"
style="border:1px solid #1cb1c2;"
allowtransparency="true" scrolling="no" class="wistia_embed"
name="wistia_embed" allowfullscreen mozallowfullscreen webkitallowfullscreen
oallowfullscreen msallowfullscreen width="640" height="360" ></iframe>
<div>
<a target="_blank" href="https://fast.wistia.net/embed/iframe/h8cqzugdrf?seo=false">
  <img alt="Opens in new tab" src="https://files.training.databricks.com/static/images/external-link-icon-16x16.png"/>&nbsp;Watch full-screen.</a>
</div>

-sandbox
### Learning from Data

Machine learning refers to a diverse set of tools for understanding data.  More technically, **machine learning is the process of _learning from data_ without being _explicitly programmed_**.  Let's unpack what that means.

Take a dataset of Boston home values in the 1970's for example.  The dataset consists of the value of homes as well as the number of rooms, crime per capita, and percent of the population considered lower class.  Home value is the _output variable_, also known as the _label_.  The other variables are known as _input variables_ or _features_.

A machine learning model would _learn_ the relationship between housing price and the various features without being explicitly programmed.  In more technical terms, our model would estimate a function `f()` that maps the relationship between input features and the output variable.

The following image shows the relation between our features and house value.  A good model `f()` would learn from the data that the number of rooms in a home is positively correlated to the house value while crime and percent of the neighborhood that is lower class is negatively correlated.  

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/boston-housing.png" style="height: 400px; margin: 20px"/></div>

The lines above represent the best fit for the data where our model's best guess for a house price given a feature value on the X axis is the corresponding point on the line.

**Machine learning is the set of approaches for estimating this function `f()` that maps features to an output.**  The inputs to this function can range from stock prices and customer information to images and DNA sequences.  Many of the same statistical techniques apply regardless of the domain.  This makes machine learning a generalizable skill set that drives decision-making in modern businesses.

-sandbox
### Getting Started

Run the following cell to configure our "classroom."

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Remember to attach your notebook to a cluster. Click <b>Detached</b> in the upper left hand corner and then select your preferred cluster.

<img src="https://s3-us-west-2.amazonaws.com/curriculum-release/images/eLearning/attach-to-cluster.png" style="border: 1px solid #aaa; border-radius: 10px 10px 10px 10px; box-shadow: 5px 5px 5px #aaa"/>

In [6]:
%run "./Includes/Classroom-Setup"

-sandbox
### Supervised vs Unsupervised Learning

Machine learning problems are roughly categorized into two main types:<br><br>

* **Supervised learning** looks to predict the value of some outcome based on one or more input measures
  - Our example of the Boston Housing Dataset is an example of supervised learning
  - In this case, the output is the price of a home and the input is features such as number of rooms
* **Unsupervised learning** describes associations and patterns in data without a known outcome
  - An example of this would be clustering customer data to find the naturally occurring customer segments
  - In this case, no known output is used as an input.  Instead, the goal is to discover how the data are organized into natural segments or clusters

This course will cover supervised learning, which is the vast majority of machine learning use cases in industry.  Later courses will look at unsupervised approaches.

<div><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/regression.png" style="height: 400px; margin: 20px"/><img src="https://files.training.databricks.com/images/eLearning/ML-Part-1/clustering.png" style="height: 400px; margin: 20px"/></div>

Import the Boston dataset, which contains median house values in 1000's (`medv`) for a variety of different features.  Since this dataset is "supervised" by the median value, this is a supervised machine learning use case.

In [9]:
bostonDF = (spark.read
  .option("HEADER", True)
  .option("inferSchema", True)
  .csv("/mnt/training/bostonhousing/bostonhousing/bostonhousing.csv")
)

display(bostonDF)

_c0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.9,19.15,27.1
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5
10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.1,18.9


-sandbox
Spark differs from many other machine learning frameworks in that we train our model on a single column that contains a vector of all of our features.  Prepare the data by creating one column named `features` that has the average number of rooms, crime rate, and poverty percentage.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the <a href="http://spark.apache.org/docs/latest/api/python/pyspark.ml.html?highlight=vectorassembler#pyspark.ml.feature.VectorAssembler" target="_blank">`VectorAssembler` documentation for more details.</a>
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> `VectorAssembler` is a tranformer, which implements a `.transform()` method.  Estimators, by contrast, need to learn from the data using a `.fit()` method before they can transform data.

In [11]:
from pyspark.ml.feature import VectorAssembler

# create feature columns
featurecols=["rm","crim","lstat"]
# create VectorAssembler using feature columns and features as output Column
assembler = VectorAssembler(inputCols=featurecols, outputCol="features")
# apply .transform() to the data
bostonFeaturizedDF = assembler.transform(bostonDF)

display(bostonFeaturizedDF)

_c0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,features
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0,"List(1, 3, List(), List(6.575, 0.00632, 4.98))"
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6,"List(1, 3, List(), List(6.421, 0.02731, 9.14))"
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,"List(1, 3, List(), List(7.185, 0.02729, 4.03))"
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,"List(1, 3, List(), List(6.998, 0.03237, 2.94))"
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2,"List(1, 3, List(), List(7.147, 0.06905, 5.33))"
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222,18.7,394.12,5.21,28.7,"List(1, 3, List(), List(6.43, 0.02985, 5.21))"
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9,"List(1, 3, List(), List(6.012, 0.08829, 12.43))"
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.9,19.15,27.1,"List(1, 3, List(), List(6.172, 0.14455, 19.15))"
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5,"List(1, 3, List(), List(5.631, 0.21124, 29.93))"
10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.1,18.9,"List(1, 3, List(), List(6.004, 0.17004, 17.1))"


In [12]:
# LinearRegression is in Spark ML library
from pyspark.ml.regression import LinearRegression
# labelCol is "medv" and featuresCol named "features")
lr=LinearRegression(labelCol="medv",featuresCol="features")

Fit the model to the data using the `.fit()` method.  This returns a trained model that has learned from our data and will be stored in `lrModel`

In [14]:
# use fit() method to create a trained model based on the boston dataset 
lrModel = lr.fit(bostonFeaturizedDF)

Take a look at what the model learned from our data.  In the case of linear regression, this comes in the form of an equation that describes the relationship the model has just learned.  This will be covered in more detail in later lessons.

In [16]:
print("Coefficients: {0:.1f}, {1:.1f}, {2:.1f}".format(*lrModel.coefficients))
print("Intercept: {0:.1f}".format(lrModel.intercept))

Coefficients: 5.2, -0.1, -0.6
Intercept: -2.6

The coefficients of our model are a multiplier against our original features and the intercept is a constant.  To interpret our model, use the following equation:

&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x number of rooms) - (.1 x crime rate) - (.6 x % lower class) - 2.6`

## Prediction

So far, we've accomplished the following:<br><br>

* Prepared data for a machine learning model
* Trained a linear regression model
* Interpreted what it learned about our data

Now, let's see how it predicts on data it has already seen as well as new data.

Create a subset of the first 10 rows of the `features` and `medv` target variable.

In [20]:
subsetDF = (bostonFeaturizedDF
  .limit(10)
  .select("features", "medv")
)

display(subsetDF)

features,medv
"List(1, 3, List(), List(6.575, 0.00632, 4.98))",24.0
"List(1, 3, List(), List(6.421, 0.02731, 9.14))",21.6
"List(1, 3, List(), List(7.185, 0.02729, 4.03))",34.7
"List(1, 3, List(), List(6.998, 0.03237, 2.94))",33.4
"List(1, 3, List(), List(7.147, 0.06905, 5.33))",36.2
"List(1, 3, List(), List(6.43, 0.02985, 5.21))",28.7
"List(1, 3, List(), List(6.012, 0.08829, 12.43))",22.9
"List(1, 3, List(), List(6.172, 0.14455, 19.15))",27.1
"List(1, 3, List(), List(5.631, 0.21124, 29.93))",16.5
"List(1, 3, List(), List(6.004, 0.17004, 17.1))",18.9


-sandbox
Use the `transform` method on the trained model to see its prediction.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Now that `lrModel` is a trained estimator, we can transform data using its `.transform()` method.

In [22]:
predictionDF = lrModel.transform(subsetDF)

display(predictionDF)

features,medv,prediction
"List(1, 3, List(), List(6.575, 0.00632, 4.98))",24.0,28.85771764778443
"List(1, 3, List(), List(6.421, 0.02731, 9.14))",21.6,25.645644850540148
"List(1, 3, List(), List(7.185, 0.02729, 4.03))",34.7,32.58746300992211
"List(1, 3, List(), List(6.998, 0.03237, 2.94))",33.4,32.241919042756436
"List(1, 3, List(), List(7.147, 0.06905, 5.33))",36.2,31.63288834584223
"List(1, 3, List(), List(6.43, 0.02985, 5.21))",28.7,27.96578524616711
"List(1, 3, List(), List(6.012, 0.08829, 12.43))",22.9,21.60241460459669
"List(1, 3, List(), List(6.172, 0.14455, 19.15))",27.1,18.543911230275786
"List(1, 3, List(), List(5.631, 0.21124, 29.93))",16.5,9.478596352794195
"List(1, 3, List(), List(6.004, 0.17004, 17.1))",18.9,18.85073477002384


How did the model do?  While there's a difference between the prediction and the true value (also known as the model error), generally the model seems to have learned something about our data.

Now predict off of a hypothetical data point of a 6 bedroom home with a 3.6 crime rate and 12 % average lower class.  According to our formula, the model should predict about 21:

&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x number of rooms) - (.1 x crime rate) - (.6 x % lower class) - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = (5.2 x 6) - (.1 x 3.6) - (.6 x 12) - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = 31.2 - .4 - 7.2 - 2.6`<br>
&nbsp;&nbsp;&nbsp;&nbsp;`predicted home value = 21`<br>

In [24]:
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([6.,3.6,12.]),)]# Creates our hypothetical data point
predictDF = spark.createDataFrame(data, ["features"])#TODO

display(lrModel.transform(predictDF))

features,prediction
"List(1, 3, List(), List(6.0, 3.6, 12.0))",21.427061506649366


## Exercise: Train a Model and Create Predictions

Train a model using the Boston dataset and different features than those used above.  Predict on new data.

### Step 1: Create the Features

Using `bostonDF`, use a `VectorAssembler` object `assembler` to create a new column `newFeatures` that has the following three variables:<br><br>

1. `indus`: proportion of non-retail business acres per town
2. `age`: proportion of owner-occupied units built prior to 1940
3. `dis`: weighted distances to five Boston employment centers

Save the results to `bostonFeaturizedDF2`

In [27]:
# TODO
from pyspark.ml.feature import VectorAssembler

featurecols=["indus","age","dis"]
assembler = VectorAssembler(inputCols=featurecols,outputCol="newFeatures")
bostonFeaturizedDF2 = assembler.transform(bostonDF)

display(bostonFeaturizedDF2)

_c0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv,newFeatures
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24.0,"List(1, 3, List(), List(2.31, 65.2, 4.09))"
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.9,9.14,21.6,"List(1, 3, List(), List(7.07, 78.9, 4.9671))"
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7,"List(1, 3, List(), List(7.07, 61.1, 4.9671))"
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4,"List(1, 3, List(), List(2.18, 45.8, 6.0622))"
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.9,5.33,36.2,"List(1, 3, List(), List(2.18, 54.2, 6.0622))"
6,0.02985,0.0,2.18,0,0.458,6.43,58.7,6.0622,3,222,18.7,394.12,5.21,28.7,"List(1, 3, List(), List(2.18, 58.7, 6.0622))"
7,0.08829,12.5,7.87,0,0.524,6.012,66.6,5.5605,5,311,15.2,395.6,12.43,22.9,"List(1, 3, List(), List(7.87, 66.6, 5.5605))"
8,0.14455,12.5,7.87,0,0.524,6.172,96.1,5.9505,5,311,15.2,396.9,19.15,27.1,"List(1, 3, List(), List(7.87, 96.1, 5.9505))"
9,0.21124,12.5,7.87,0,0.524,5.631,100.0,6.0821,5,311,15.2,386.63,29.93,16.5,"List(1, 3, List(), List(7.87, 100.0, 6.0821))"
10,0.17004,12.5,7.87,0,0.524,6.004,85.9,6.5921,5,311,15.2,386.71,17.1,18.9,"List(1, 3, List(), List(7.87, 85.9, 6.5921))"


In [28]:
# TEST - Run this cell to test your solution
dbTest("ML1-P-02-01-01", True, set(assembler.getInputCols()) == {'indus', 'age', 'dis'})
dbTest("ML1-P-02-01-02", True, bool(bostonFeaturizedDF2.schema['newFeatures'].dataType))

print("Tests passed!")

Tests passed!

### Step 2: Train the Model

Instantiate a linear regression model `lrNewFeatures`.  Save the trained model to `lrModelNew`.

In [30]:
# TODO
from pyspark.ml.regression import LinearRegression

lrNewFeatures = LinearRegression(labelCol="medv",featuresCol="newFeatures")
lrModelNew = lrNewFeatures.fit(bostonFeaturizedDF2)

In [31]:
# TEST - Run this cell to test your solution
dbTest("ML1-P-02-02-01", True, lrNewFeatures.getFeaturesCol() == "newFeatures")
dbTest("ML1-P-02-02-02", True, lrNewFeatures.getLabelCol() == "medv")
dbTest("ML1-P-02-02-03", True, lrModelNew.hasSummary)

print("Tests passed!")

Tests passed!

### Step 3: Create Predictions

Create the DataFrame `predictionsDF` for the following values, created for you in `newDataDF`:

| Feature | Datapoint 1 | Datapoint 2 | Datapoint 3 |
|:--------|:------------|:------------|:------------|
| `indus` | 11          | 6           | 19          |
| `age`   | 68          | 35          | 74          |
| `dis`   | 4           | 2           | 8           |

In [33]:
# TODO
from pyspark.ml.linalg import Vectors

data = [(Vectors.dense([11., 68., 4.]), ),
        (Vectors.dense([6., 35., 2.]), ),
        (Vectors.dense([19., 74., 8.]), )]
newDataDF = spark.createDataFrame(data, ["newFeatures"])
predictionsDF = lrModelNew.transform(newDataDF)

display(predictionsDF)

newFeatures,prediction
"List(1, 3, List(), List(11.0, 68.0, 4.0))",22.370810825866744
"List(1, 3, List(), List(6.0, 35.0, 2.0))",32.24076584405395
"List(1, 3, List(), List(19.0, 74.0, 8.0))",9.742860699127554


In [34]:
# TEST - Run this cell to test your solution
predicitions = [i.prediction for i in predictionsDF.select("prediction").collect()]

dbTest("ML1-P-02-02-01", True, predicitions[0] > 20 and predicitions[0] < 23)
dbTest("ML1-P-02-02-01", True, predicitions[1] > 30 and predicitions[1] < 34)
dbTest("ML1-P-02-02-01", True, predicitions[2] > 7 and predicitions[2] < 11)

print("Tests passed!")

Tests passed!

## Review

**Question:** If machine learning has to do with learning from data without being explicitly programmed, how do the algorithms learn?  
**Answer:** Machine learning uses various objective metrics that the algorithm is trying to minimize or maximize.  For instance, in the case of the housing dataset we're penalizing the algorithm for predicting a value that's far from the true value.  This gives the algorithm an incentive to perform well.  Once we know what we want to minimize, calculus and linear algebra are used in order to accomplish the best way to minimize that value.

**Question:** In "supervised" learning, what is doing the supervising?  
**Answer:** Supervised machine learning problems are "supervised" by a known output (also known as the data's label).  In essence, the algorithm is modeling the relationship between the input features and that output.  The output of a model is also known as a teacher, supervisor, dependent variable, or simply `y`.  In the case of unsupervised algorithms, there is no dependent variable to supervise the learning process.

**Question:** Why does the difference between regression and classification matter?  
**Answer:** The way we model continuous and categorical variables works quite differently.  These differences results in the field of discrete math dedicated to the study of discrete values.  If we're modeling US states, for instance, we could assign the value of 1 to be California and 2 to be Washington.  There's no real way that Washington is twice California.  Rather, these numbers represent distinct groups within our data.  Supervised machine learning problems are called regression problems when the outcome we're modeling is continuous and classification when the outcome is categorical.  As we'll explore in the lesson on featurization, the way we handle categorical variables when they appear in our features is different as well.

**Question:** What's the difference between the `fit` and `transform` methods on a machine learning model?  
**Answer:** The `fit` method learns something from the data and saves the result in the model object (e.g. `lrModel` above).  The `transform` method applies the learned pattern to unseen data.  `transform` can be used to make predictions using a trained model or to change the data in some other way, such as encoding a categorical feature.  We'll explore this more in later lessons.

## Next Steps

Start the next lesson, [Exploratory Analysis]($./03-Exploratory-Analysis ).

## Additional Topics & Resources

**Q:** What is a good general resource for machine learning?  
**A:** <a href="https://www-bcf.usc.edu/~gareth/ISL/" target="_blank">_An Introduction to Statistical Learning_</a> is a good starting point for the themes and basic approaches to machine learning.

**Q:** How can I use `sklearn` and other common machine learning tools with Databricks?  
**A:** `sklearn` comes pre-installed on Databricks clusters.  Additional libraries can be installed as needed.  Also, R code can be run in the same notebook by including `%r` in the first line of a code cell.

**Q:** How can I combine my `sklearn` code and the distributed nature of Spark?  
**A:** One common machine learning task is to perform gridsearch in order to choose the optimal model hyperparameters.  This can be easily parallelized on Spark <a href="https://github.com/databricks/spark-sklearn" target="_blank">using the `spark-sklearn` package.</a>

**Q:** Where can I find out more information on machine learning with Spark?  
**A:** Check out the Databricks blog <a href="https://databricks.com/blog/category/engineering/machine-learning" target="_blank">dedicated to machine learning</a>

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>